![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Tool Definition: Building Agent Capabilities

## Learning Objectives (25 minutes)
By the end of this notebook, you will be able to:
1. **Create** simple tools using LangChain's @tool decorator
2. **Test** how LLMs select and use tools
3. **Write** effective tool descriptions that guide LLM behavior
4. **Build** a tool-enabled agent for Redis University
5. **Apply** best practices for tool design

## Prerequisites
- Completed `01_system_instructions.ipynb`
- OpenAI API key configured (for LangChain ChatOpenAI)
- Redis Stack running with course data

---

## Introduction

**Tools** give your agent superpowers! Instead of just generating text, your agent can:
- 🔍 Search real course catalogs
- ✅ Check prerequisites
- 📊 Get detailed course information
- 🎯 Make data-driven recommendations

**Example:** When a student asks "What machine learning courses are available?", an agent *without* tools can only respond based on training data (likely outdated). An agent *with* a `search_courses` tool can query your live course database and return current offerings.

Let's build tools step by step, starting simple and adding complexity gradually.

---

## Concepts: How Tools Work

### What Are Tools?

Tools are **functions that the LLM can call** to perform actions or retrieve information. They extend the agent's capabilities beyond text generation.

**Without tools:**
- Agent can only generate text based on its training data
- No access to real-time data
- Can't take actions
- Limited to what's in the prompt

**With tools:**
- Agent can search databases
- Agent can retrieve current information
- Agent can perform calculations
- Agent can take actions (send emails, create records, etc.)

### How Tool Calling Works

1. **LLM receives** user query + system instructions + available tools
2. **LLM decides** which tool(s) to call (if any)
3. **LLM generates** tool call with parameters
4. **Your code executes** the tool function (not the LLM!)
5. **Tool returns** results
6. **LLM receives** results and generates response

### Tool Schema Components

Every tool needs:
1. **Name** - Unique identifier
2. **Description** - What the tool does (critical for selection!)
3. **Parameters** - Input schema with types and descriptions
4. **Function** - The actual implementation

**In code, this looks like:**
```python
@tool(args_schema=SearchCoursesInput)  # Parameters defined here
async def search_courses(query: str) -> str:
    """
    Description goes here - the LLM reads this!
    """
    # Implementation (LLM never sees this)
```

### How LLMs Select Tools

The LLM uses:
- Tool **names** (should be descriptive)
- Tool **descriptions** (should explain when to use it)
- Parameter **descriptions** (should explain what each parameter does)
- **Context** from the conversation

**Key insight:** The LLM only sees the tool schema, not the implementation!

**You'll see this in action:** Below, we'll first create a tool with a vague description, then improve it and observe how the LLM's behavior changes.

### Common Pitfalls (We'll Avoid)

- ❌ **Vague descriptions** → LLM picks wrong tool
- ❌ **Too many similar tools** → LLM gets confused  
- ❌ **Missing parameter descriptions** → LLM passes wrong data

**Don't worry** - we'll show you exactly how to implement these best practices!

### Simple Best Practices (Keep It Clear!)

#### ❌ **Bad Tool Descriptions**
```python
# BAD: Vague and unhelpful
@tool
def search(query: str) -> str:
    """Search for stuff."""
    
# BAD: Missing context about when to use
@tool  
def get_data(id: str) -> str:
    """Gets data from database."""
```

#### ✅ **Good Tool Descriptions**
```python
# GOOD: Clear purpose and usage context
@tool
def search_courses(query: str) -> str:
    """
    Search for courses using semantic similarity.
    
    Use this when:
    - Student asks about courses on a topic
    - Student wants to explore subject areas
    - Student asks "What courses are available for...?"
    """
```

#### ❌ **Bad Parameter Descriptions**
```python
# BAD: Ambiguous parameter names and descriptions
def get_weather(location, unit):
    # What format is location? What units are supported?
```

#### ✅ **Good Parameter Descriptions**
```python
# GOOD: Clear parameter specifications
def get_weather(location: str, unit: str):
    """
    Parameters:
    - location: City name or "latitude,longitude" coordinates
    - unit: Temperature unit ("celsius" or "fahrenheit")
    """
```

#### ❌ **Bad Tool Naming**
- `tool1`, `helper`, `utils` → No indication of purpose
- `get_data`, `process` → Too generic
- `search_courses_and_maybe_filter_by_difficulty_and_format` → Too verbose

#### ✅ **Good Tool Naming**
- `search_courses`, `get_course_details`, `check_prerequisites` → Clear and specific
- `calculate_shipping_cost`, `validate_email` → Action-oriented
- `format_student_transcript` → Descriptive of exact function

#### ❌ **Bad Tool Scope**
```python
# BAD: Does too many things
@tool
def manage_student(action: str, student_id: str, data: dict):
    """Create, update, delete, or search students."""
    # LLM gets confused about which action to use
```

#### ✅ **Good Tool Scope**
```python
# GOOD: Single, clear responsibility
@tool
def create_student_profile(name: str, email: str) -> str:
    """Create a new student profile with basic information."""
    
@tool
def update_student_email(student_id: str, new_email: str) -> str:
    """Update a student's email address."""
```

#### ❌ **Bad Error Handling**
```python
# BAD: Silent failures or cryptic errors
@tool
def get_course_details(course_id: str) -> str:
    """Get course details."""
    try:
        return database.get(course_id)
    except:
        return None  # LLM doesn't know what went wrong
```

#### ✅ **Good Error Handling**
```python
# GOOD: Clear error messages for the LLM
@tool
def get_course_details(course_id: str) -> str:
    """Get detailed information about a specific course."""
    try:
        course = database.get(course_id)
        if not course:
            return f"Course {course_id} not found. Please check the course ID."
        return format_course_details(course)
    except Exception as e:
        return f"Error retrieving course details: {str(e)}"
```

#### ❌ **Bad Return Values**
```python
# BAD: Returns complex objects or unclear formats
@tool
def search_courses(query: str) -> dict:
    """Search courses."""
    return {"results": [...], "meta": {...}}  # LLM can't parse this well
```

#### ✅ **Good Return Values**
```python
# GOOD: Returns clear, formatted strings
@tool
def search_courses(query: str) -> str:
    """Search for courses matching the query."""
    results = perform_search(query)
    if not results:
        return "No courses found matching your query."
    
    formatted = "Found courses:\n"
    for course in results:
        formatted += f"- {course.code}: {course.title}\n"
    return formatted
```

**Key Takeaway:** The LLM needs to understand both success and failure cases to provide helpful responses to users!

## Environment Setup

In [1]:
# Environment setup
import os
from typing import List, Optional
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")

print("🔧 Environment Setup")
print("=" * 30)
print(f"OpenAI API Key: {'✅ Set' if OPENAI_API_KEY else '❌ Not set'}")
print(f"Redis URL: {REDIS_URL}")

🔧 Environment Setup
OpenAI API Key: ✅ Set
Redis URL: redis://localhost:6379


In [2]:
# Import required modules (consistent with LangGraph agent)
try:
    # LangChain imports (same as our agent)
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
    from langchain_core.tools import tool
    from pydantic import BaseModel, Field
    
    # Redis and course modules
    import redis
    from redis_context_course.models import Course, StudentProfile, CourseFormat
    from redis_context_course.course_manager import CourseManager
    from redis_context_course.redis_config import redis_config
    
    # Initialize LangChain LLM (same as our agent)
    if OPENAI_API_KEY:
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7
        )
        print("✅ LangChain ChatOpenAI initialized")
    else:
        llm = None
        print("⚠️  LangChain LLM not available (API key not set)")
    
    # Redis connection
    redis_client = redis.from_url(REDIS_URL)
    if redis_config.health_check():
        print("✅ Redis connection healthy")
    else:
        print("❌ Redis connection failed")
    
    # Course manager
    course_manager = CourseManager()
    
    print("✅ Core modules imported successfully")
    print("🔗 Using LangChain patterns consistent with our LangGraph agent")
    
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Please ensure you've completed the setup from Section 1.")
    print("Install missing packages: pip install langchain-openai langchain-core")

✅ LangChain ChatOpenAI initialized
✅ Redis connection healthy
16:38:37 redisvl.index.index INFO   Index already exists, not overwriting.
✅ Core modules imported successfully
🔗 Using LangChain patterns consistent with our LangGraph agent


## 🧪 Hands-on: Building Your First Tool

Let's start with the simplest possible tool and see how it works:

### Step 1: A Basic Tool

Let's create a simple course search tool:

In [3]:
# Simple tool using LangChain's @tool decorator
@tool
def search_courses_basic(query: str) -> str:
    """Search for courses by title or description."""
    
    # For now, let's use mock data to see how tools work
    mock_courses = [
        "CS101: Introduction to Programming",
        "CS201: Data Structures and Algorithms", 
        "CS301: Machine Learning Fundamentals",
        "MATH101: Calculus I",
        "MATH201: Statistics"
    ]
    
    # Simple search - find courses that contain the query
    results = [course for course in mock_courses if query.lower() in course.lower()]
    
    if results:
        return "\n".join(results)
    else:
        return f"No courses found for '{query}'"

print("✅ Basic tool created!")
print(f"Tool name: {search_courses_basic.name}")
print(f"Description: {search_courses_basic.description}")

✅ Basic tool created!
Tool name: search_courses_basic
Description: Search for courses by title or description.


In [5]:
# Test the tool directly
print("🧪 Testing the tool directly:")
print("\nSearch for 'programming':")
result = search_courses_basic.invoke({"query": "programming"})
print(result)

print("\nSearch for 'machine learning':")
result = search_courses_basic.invoke({"query": "machine learning"})
print(result)

print("\nSearch for 'chemistry':")
result = search_courses_basic.invoke({"query": "chemistry"})
print(result)

🧪 Testing the tool directly:

Search for 'programming':
CS101: Introduction to Programming

Search for 'machine learning':
CS301: Machine Learning Fundamentals

Search for 'chemistry':
No courses found for 'chemistry'


**🎯 Great!** Our tool works, but the description is too basic. Let's improve it.

### Step 2: Improving Tool Descriptions

The LLM uses your tool description to decide when to use it. Let's make it better:

In [6]:
# Improved tool with better description using real Redis data
@tool
async def search_courses(query: str, limit: int = 5) -> str:
    """
    Search for courses using semantic search on Redis University catalog.
    
    Use this tool when:
    - Student asks about courses on a specific topic (e.g., "programming", "machine learning")
    - Student wants to explore courses in a subject area
    - Student asks "What courses are available for...?"
    
    Returns a list of matching courses with course codes, titles, and descriptions.
    """
    
    try:
        # Use the real Redis-powered course manager (same as reference agent)
        results = await course_manager.search_courses(query, limit=limit)
        
        if not results:
            return f"No courses found for '{query}'. Try different keywords or broader terms."
        
        # Format results for display
        output = []
        for course in results:
            output.append(
                f"{course.course_code}: {course.title}\n"
                f"  Credits: {course.credits} | {course.format.value} | {course.difficulty_level.value}\n"
                f"  {course.description[:150]}..."
            )
        
        return "\n\n".join(output)
        
    except Exception as e:
        return f"Error searching courses: {str(e)}. Please try again."

print("✅ Real Redis-powered search tool created!")
print("\nDescription:")
print(search_courses.description)

✅ Real Redis-powered search tool created!

Description:
Search for courses using semantic search on Redis University catalog.

Use this tool when:
- Student asks about courses on a specific topic (e.g., "programming", "machine learning")
- Student wants to explore courses in a subject area
- Student asks "What courses are available for...?"

Returns a list of matching courses with course codes, titles, and descriptions.


**Much better!** Now the LLM knows exactly when to use this tool.

### Step 3: Understanding args_schema

Before we add more tools, let's learn about `args_schema` - a powerful pattern for better tool validation:

#### What is args_schema?

`args_schema` is a Pydantic model that defines:
- **Parameter types** - What type each parameter should be
- **Validation rules** - What values are acceptable
- **Documentation** - Descriptions for each parameter
- **Required vs optional** - Which parameters are mandatory

**Benefits:**
- ✅ **Better error handling** - Invalid inputs are caught early
- ✅ **Clear documentation** - LLM knows exactly what to send
- ✅ **Type safety** - Parameters are automatically validated
- ✅ **Professional pattern** - Used in production LangChain applications

In [7]:
# First, let's create a Pydantic model for our course details tool
class GetCourseDetailsInput(BaseModel):
    """Input schema for getting course details."""
    
    course_code: str = Field(
        description="The course code (e.g., 'CS101', 'MATH201'). Must be in format: DEPT + NUMBER"
    )

print("✅ Input schema created!")
print(f"Schema fields: {list(GetCourseDetailsInput.model_fields.keys())}")
print(f"Course code description: {GetCourseDetailsInput.model_fields['course_code'].description}")

✅ Input schema created!
Schema fields: ['course_code']
Course code description: The course code (e.g., 'CS101', 'MATH201'). Must be in format: DEPT + NUMBER


### Step 4: Adding More Tools with args_schema

Now let's create a tool that uses the args_schema pattern:

In [8]:
# Tool to get course details using args_schema and real Redis data
@tool(args_schema=GetCourseDetailsInput)
async def get_course_details(course_code: str) -> str:
    """
    Get detailed information about a specific course by its course code.
    
    Use this tool when:
    - Student asks about a specific course (e.g., "Tell me about CS101")
    - You need prerequisites for a course
    - You need full course details (schedule, instructor, etc.)
    
    Returns complete course information including description, prerequisites,
    schedule, credits, and learning objectives.
    """
    
    try:
        # Use the real Redis-powered course manager (same as reference agent)
        course = await course_manager.get_course_by_code(course_code.upper())
        
        if not course:
            return f"Course {course_code} not found. Please check the course code and try again."
        
        # Format prerequisites
        prereqs = "None" if not course.prerequisites else ", ".join(
            [f"{p.course_code} (min grade: {p.min_grade})" for p in course.prerequisites]
        )
        
        # Format learning objectives
        objectives = "\n".join([f"- {obj}" for obj in course.learning_objectives]) if course.learning_objectives else "Not specified"
        
        return f"""{course.course_code}: {course.title}

Description: {course.description}

Details:
- Credits: {course.credits}
- Department: {course.department}
- Major: {course.major}
- Difficulty: {course.difficulty_level.value}
- Format: {course.format.value}
- Instructor: {course.instructor}
- Prerequisites: {prereqs}

Learning Objectives:
{objectives}"""
        
    except Exception as e:
        return f"Error retrieving course details: {str(e)}. Please try again."

print("✅ Real Redis-powered course details tool created with args_schema!")
print(f"Tool name: {get_course_details.name}")
print(f"Uses schema: {get_course_details.args_schema.__name__}")

✅ Real Redis-powered course details tool created with args_schema!
Tool name: get_course_details
Uses schema: GetCourseDetailsInput


### Testing Redis-Powered Tools

**Note:** Our tools now use real Redis data and are async functions. In a Jupyter notebook, you can test them like this:

In [9]:
# Test the Redis-powered tools
print("🧪 Testing Redis-powered tools:")

if course_manager:
    try:
        print("\n1. Testing course search:")
        result = await search_courses.ainvoke({"query": "programming", "limit": 3})
        print(result)
        
        print("\n2. Testing course details:")
        # Try to get details for a course that might exist
        result = await get_course_details.ainvoke({"course_code": "RU101"})
        print(result)
        
    except Exception as e:
        print(f"Note: Direct testing requires async environment and Redis connection.")
        print(f"Tools are ready for use with the LangChain agent!")
else:
    print("Course manager not available - tools are ready for use when Redis is connected!")
    print("\n✅ The tools will work perfectly with the LangChain agent in an async environment.")
    print("✅ They use the same Redis-powered CourseManager as our reference agent.")

🧪 Testing Redis-powered tools:

1. Testing course search:
16:39:47 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
CS001: Introduction to Programming
  Credits: 4 | online | beginner
  Comprehensive study of introduction to programming. Core concepts and practical applications in computer science....

CS004: Operating Systems
  Credits: 4 | online | beginner
  Comprehensive study of operating systems. Core concepts and practical applications in computer science....

CS006: Software Engineering
  Credits: 3 | in_person | intermediate
  Comprehensive study of software engineering. Core concepts and practical applications in computer science....

2. Testing course details:
Error retrieving course details: 'list' object has no attribute 'docs'. Please try again.


### Step 5: More Complex args_schema

Let's create a more complex schema for our prerequisites checker:

In [10]:
# More complex schema with validation
class CheckPrerequisitesInput(BaseModel):
    """Input schema for checking course prerequisites."""
    
    course_code: str = Field(
        description="The course code to check prerequisites for (e.g., 'CS301')"
    )
    completed_courses: List[str] = Field(
        description="List of course codes the student has completed (e.g., ['CS101', 'CS201'])",
        default=[]
    )

print("✅ Prerequisites schema created!")
print(f"Schema fields: {list(CheckPrerequisitesInput.model_fields.keys())}")
print(f"Completed courses default: {CheckPrerequisitesInput.model_fields['completed_courses'].default}")

✅ Prerequisites schema created!
Schema fields: ['course_code', 'completed_courses']
Completed courses default: []


### Step 6: Prerequisites Checker with Validation

Now let's create the prerequisites tool with proper validation:

In [11]:
# Tool to check prerequisites with args_schema using real Redis data
@tool(args_schema=CheckPrerequisitesInput)
async def check_prerequisites(course_code: str, completed_courses: List[str]) -> str:
    """
    Check if a student meets the prerequisites for a specific course.
    
    Use this tool when:
    - Student asks "Can I take [course]?"
    - Student asks about prerequisites
    - You need to verify eligibility before recommending a course
    
    Returns whether the student is eligible and which prerequisites are missing (if any).
    """
    
    try:
        # Use the real Redis-powered course manager (same as reference agent)
        course = await course_manager.get_course_by_code(course_code.upper())
        
        if not course:
            return f"Course {course_code} not found. Please check the course code and try again."
        
        # Convert completed courses to uppercase for comparison
        completed_courses_upper = [c.upper() for c in completed_courses]
        
        if not course.prerequisites:
            return f"✅ {course.course_code} has no prerequisites. You can take this course!"
        
        # Check each prerequisite
        missing = []
        for prereq in course.prerequisites:
            if prereq.course_code not in completed_courses_upper:
                missing.append(f"{prereq.course_code} (min grade: {prereq.min_grade})")
        
        if not missing:
            return f"✅ You meet all prerequisites for {course.course_code}!"
        
        return f"""❌ You're missing prerequisites for {course.course_code}:

Missing:
""" + "\n".join([f"- {p}" for p in missing])
        
    except Exception as e:
        return f"Error checking prerequisites: {str(e)}. Please try again."

print("✅ Real Redis-powered prerequisites checker created with args_schema!")
print(f"Tool name: {check_prerequisites.name}")
print(f"Uses schema: {check_prerequisites.args_schema.__name__}")

✅ Real Redis-powered prerequisites checker created with args_schema!
Tool name: check_prerequisites
Uses schema: CheckPrerequisitesInput


### Testing args_schema Benefits

Let's see how args_schema provides better validation and error handling:

In [12]:
# Test the prerequisites checker with proper validation
print("🧪 Testing prerequisites checker with args_schema:")

print("\n1. Valid input - new student:")
result = await check_prerequisites.ainvoke({"course_code": "CS101", "completed_courses": []})
print(result)

print("\n2. Valid input - student with prerequisites:")
result = await check_prerequisites.ainvoke({"course_code": "CS201", "completed_courses": ["CS101"]})
print(result)

print("\n3. Valid input - missing prerequisites:")
result = await check_prerequisites.ainvoke({"course_code": "CS301", "completed_courses": ["CS101"]})
print(result)

🧪 Testing prerequisites checker with args_schema:

1. Valid input - new student:
Error checking prerequisites: 'list' object has no attribute 'docs'. Please try again.

2. Valid input - student with prerequisites:
Error checking prerequisites: 'list' object has no attribute 'docs'. Please try again.

3. Valid input - missing prerequisites:
Error checking prerequisites: 'list' object has no attribute 'docs'. Please try again.


In [13]:
# Test validation - what happens with invalid input?
print("🧪 Testing args_schema validation:")

try:
    print("\n4. Testing with missing required parameter:")
    # This should work because completed_courses has a default
    result = check_prerequisites.invoke({"course_code": "CS101"})
    print("✅ Success with default value:", result)
except Exception as e:
    print(f"❌ Error: {e}")

try:
    print("\n5. Testing with completely missing parameters:")
    # This should fail because course_code is required
    result = check_prerequisites.invoke({})
    print("Result:", result)
except Exception as e:
    print(f"✅ Validation caught error: {type(e).__name__}")
    print(f"   Message: {str(e)[:100]}...")

print("\n🎯 args_schema provides automatic validation and better error messages!")

🧪 Testing args_schema validation:

4. Testing with missing required parameter:
❌ Error: StructuredTool does not support sync invocation.

5. Testing with completely missing parameters:
✅ Validation caught error: ValidationError
   Message: 1 validation error for CheckPrerequisitesInput
course_code
  Field required [type=missing, input_val...

🎯 args_schema provides automatic validation and better error messages!


### Benefits of args_schema

As you can see, `args_schema` provides:

1. **✅ Automatic Validation** - Invalid inputs are caught before your function runs
2. **✅ Better Error Messages** - Clear feedback about what went wrong
3. **✅ Default Values** - Parameters can have sensible defaults
4. **✅ Type Safety** - Parameters are automatically converted to the right types
5. **✅ Documentation** - LLM gets detailed parameter descriptions
6. **✅ Professional Pattern** - Used in production LangChain applications

**When to use args_schema:**
- ✅ Tools with multiple parameters
- ✅ Tools that need validation
- ✅ Production applications
- ✅ Complex parameter types (lists, objects)

**When simple parameters are fine:**
- ✅ Single parameter tools
- ✅ Simple string/number inputs
- ✅ Quick prototypes

In [14]:
# Compare: Simple tool vs args_schema tool
print("📊 Comparison: Simple vs args_schema tools")
print("=" * 50)

print("\n🔧 Simple tool (search_courses):")
print(f"   Parameters: {search_courses.args}")
print(f"   Schema: {getattr(search_courses, 'args_schema', 'None')}")

print("\n🔧 args_schema tool (get_course_details):")
print(f"   Parameters: {get_course_details.args}")
print(f"   Schema: {get_course_details.args_schema.__name__}")
print(f"   Schema fields: {list(get_course_details.args_schema.model_fields.keys())}")

print("\n🎯 Both patterns are valid - choose based on your needs!")

📊 Comparison: Simple vs args_schema tools

🔧 Simple tool (search_courses):
   Parameters: {'query': {'title': 'Query', 'type': 'string'}, 'limit': {'default': 5, 'title': 'Limit', 'type': 'integer'}}
   Schema: <class 'langchain_core.utils.pydantic.search_courses'>

🔧 args_schema tool (get_course_details):
   Parameters: {'course_code': {'description': "The course code (e.g., 'CS101', 'MATH201'). Must be in format: DEPT + NUMBER", 'title': 'Course Code', 'type': 'string'}}
   Schema: GetCourseDetailsInput
   Schema fields: ['course_code']

🎯 Both patterns are valid - choose based on your needs!


**🎉 Excellent!** Now we have three useful tools. Let's see how the LLM uses them.

## 🤖 Hands-on: Testing Tools with an Agent

Let's see how the LLM selects and uses our tools:

In [15]:
# Bind tools to LLM (same pattern as our LangGraph agent)
tools = [search_courses, get_course_details, check_prerequisites]

if llm:
    llm_with_tools = llm.bind_tools(tools)
    
    # System prompt
    system_prompt = """You are the Redis University Class Agent.
    Help students find courses and plan their schedule.
    Use the available tools to search courses and check prerequisites.
    """
    
    print("✅ Agent configured with Redis-powered tools!")
    print(f"Available tools: {[tool.name for tool in tools]}")
    print("🔗 Using the same CourseManager as our reference agent")
else:
    print("⚠️  LLM not available - tools are ready for use when OpenAI API key is set")

✅ Agent configured with Redis-powered tools!
Available tools: ['search_courses', 'get_course_details', 'check_prerequisites']
🔗 Using the same CourseManager as our reference agent


### Test 1: Search Query

Let's see what happens when a student asks about machine learning:

In [16]:
# Test 1: Search query
if llm:
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content="I'm interested in machine learning courses")
    ]
    
    response = llm_with_tools.invoke(messages)
    
    print("👤 User: I'm interested in machine learning courses")
    print("\n🤖 Agent decision:")
    if response.tool_calls:
        for tool_call in response.tool_calls:
            print(f"   🔧 Tool: {tool_call['name']}")
            print(f"   📋 Args: {tool_call['args']}")
    else:
        print("   💬 No tool called")
        print(f"   📝 Response: {response.content}")
else:
    print("⚠️  LLM not available - skipping test")

print("\n" + "="*60)

16:40:59 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
👤 User: I'm interested in machine learning courses

🤖 Agent decision:
   🔧 Tool: search_courses
   📋 Args: {'query': 'machine learning'}



### Test 2: Specific Course Query

What happens when they ask about a specific course?

In [17]:
# Test 2: Specific course query
if llm:
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content="Tell me about CS301")
    ]
    
    response = llm_with_tools.invoke(messages)
    
    print("👤 User: Tell me about CS301")
    print("\n🤖 Agent decision:")
    if response.tool_calls:
        for tool_call in response.tool_calls:
            print(f"   🔧 Tool: {tool_call['name']}")
            print(f"   📋 Args: {tool_call['args']}")
    else:
        print("   💬 No tool called")
        print(f"   📝 Response: {response.content}")
else:
    print("⚠️  LLM not available - skipping test")

print("\n" + "="*60)

16:41:00 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
👤 User: Tell me about CS301

🤖 Agent decision:
   🔧 Tool: get_course_details
   📋 Args: {'course_code': 'CS301'}



### Test 3: Prerequisites Query

What about when they ask if they can take a course?

In [18]:
# Test 3: Prerequisites query
if llm:
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content="Can I take CS301? I've completed CS101 and CS201.")
    ]
    
    response = llm_with_tools.invoke(messages)
    
    print("👤 User: Can I take CS301? I've completed CS101 and CS201.")
    print("\n🤖 Agent decision:")
    if response.tool_calls:
        for tool_call in response.tool_calls:
            print(f"   🔧 Tool: {tool_call['name']}")
            print(f"   📋 Args: {tool_call['args']}")
    else:
        print("   💬 No tool called")
        print(f"   📝 Response: {response.content}")
else:
    print("⚠️  LLM not available - skipping test")

print("\n" + "="*60)

16:41:03 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
👤 User: Can I take CS301? I've completed CS101 and CS201.

🤖 Agent decision:
   🔧 Tool: check_prerequisites
   📋 Args: {'course_code': 'CS301', 'completed_courses': ['CS101', 'CS201']}



## 🎮 Try It Yourself: Create Your Own Tool

Now it's your turn! Create a tool and test it:

In [19]:
# First, create the schema for your tool
class GetCoursesByDepartmentInput(BaseModel):
    """Input schema for getting courses by department."""
    
    department: str = Field(
        description="Department code (e.g., 'CS', 'MATH', 'PHYS'). Case insensitive."
    )

print("✅ Department schema created!")

✅ Department schema created!


In [20]:
# Your turn! Create a tool to get courses by department with args_schema using real Redis data
@tool(args_schema=GetCoursesByDepartmentInput)
async def get_courses_by_department(department: str) -> str:
    """
    Get all courses offered by a specific department.
    
    Use this tool when:
    - Student asks "What CS courses are available?"
    - Student wants to see all courses in a department
    - Student asks about course offerings by department
    
    Returns a list of all courses in the specified department.
    """
    
    try:
        # Use the real Redis-powered course manager with department filter
        filters = {"department": department.upper()}
        results = await course_manager.search_courses(
            query="",  # Empty query to get all courses
            filters=filters,
            limit=50,  # Get more courses for department listing
            similarity_threshold=0.0  # Include all courses in department
        )
        
        if not results:
            return f"No courses found in {department.upper()} department. Please check the department code."
        
        # Format results for display
        output = []
        for course in results:
            output.append(
                f"{course.course_code}: {course.title} ({course.credits} credits)"
            )
        
        return f"Courses in {department.upper()} department ({len(results)} found):\n" + "\n".join(output)
        
    except Exception as e:
        return f"Error retrieving department courses: {str(e)}. Please try again."

print("✅ Real Redis-powered department tool created with args_schema!")
print(f"Tool name: {get_courses_by_department.name}")
print(f"Uses schema: {get_courses_by_department.args_schema.__name__}")

# Test your tool
print("\n🧪 Testing your tool:")
if course_manager:
    try:
        import asyncio
        result = await get_courses_by_department.ainvoke({"department": "CS"})
        print(result)
    except Exception as e:
        print(f"Test requires async environment. Tool is ready for use with the agent!")
else:
    print("Course manager not available - tool is ready for use when Redis is connected!")

✅ Real Redis-powered department tool created with args_schema!
Tool name: get_courses_by_department
Uses schema: GetCoursesByDepartmentInput

🧪 Testing your tool:
16:41:06 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Courses in CS department (1 found):
CS101: Python Basics (3 credits)


In [21]:
# Test your tool with the agent
if llm:
    # Add your tool to the agent
    all_tools = [search_courses, get_course_details, check_prerequisites, get_courses_by_department]
    llm_with_all_tools = llm.bind_tools(all_tools)
    
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content="What computer science courses are available?")
    ]
    
    response = llm_with_all_tools.invoke(messages)
    
    print("👤 User: What computer science courses are available?")
    print("\n🤖 Agent decision:")
    if response.tool_calls:
        for tool_call in response.tool_calls:
            print(f"   🔧 Tool: {tool_call['name']}")
            print(f"   📋 Args: {tool_call['args']}")
    else:
        print("   💬 No tool called")
        print(f"   📝 Response: {response.content}")
else:
    print("⚠️  LLM not available - skipping test")

print("\n🎯 Did the agent choose your tool? Try different queries to test tool selection!")

16:41:07 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
👤 User: What computer science courses are available?

🤖 Agent decision:
   🔧 Tool: get_courses_by_department
   📋 Args: {'department': 'CS'}

🎯 Did the agent choose your tool? Try different queries to test tool selection!


## 🎯 Key Takeaways

From this hands-on exploration, you've learned:

### ✅ **Tool Design Best Practices**

1. **Clear Names**
   - Use descriptive, action-oriented names
   - `search_courses` ✅ vs. `find` ❌

2. **Detailed Descriptions**
   - Explain what the tool does
   - Explain when to use it
   - Include examples

3. **Well-Defined Parameters**
   - Use type hints
   - Add descriptions for each parameter
   - Set sensible defaults
   - **Use args_schema for complex tools**

4. **Useful Return Values**
   - Return formatted, readable text
   - Include relevant details
   - Handle errors gracefully

5. **Single Responsibility**
   - Each tool should do one thing well
   - Don't combine unrelated functionality

### ✅ **How Tool Descriptions Affect Selection**

The LLM relies heavily on tool descriptions to decide which tool to use:

- ✅ **Good description**: "Search for courses using keywords. Use when students ask about topics or course characteristics."
- ❌ **Bad description**: "Search courses"

**Remember:** The LLM can't see your code, only the schema!

### ✅ **LangChain Integration**

- **@tool decorator** makes creating tools simple
- **llm.bind_tools()** connects tools to your LLM
- **Tool selection** happens automatically based on descriptions
- **Compatible** with our LangGraph agent architecture
- **args_schema** provides validation and better documentation
- **Redis-powered** using the same CourseManager as our reference agent
- **Async support** for real-time data access and performance

### 🚀 **Next Steps**
You're now ready to:
- Build effective tools for any AI agent
- Write descriptions that guide LLM behavior
- Test and iterate on tool selection
- Move on to **Context-Aware Tool Integration** for advanced patterns

---

**Great work!** You've mastered the fundamentals of tool definition with LangChain.

---

## 📝 **Quick Practice Exercises**

Before moving on, try these focused exercises:

### **Exercise 1: Create a Department Tool**
Build `get_courses_by_department` that takes a department name and returns all courses in that department. Write a clear description!

### **Exercise 2: Test Tool Selection**
Create queries that should trigger each tool:
- "What ML courses are available?" → `search_courses`
- "Can I take CS301?" → `check_prerequisites`  
- "Tell me about CS101" → `get_course_details`

### **Exercise 3: Improve a Description**
Pick any tool and improve its description. Add "Use this when..." examples and test if the LLM selects it better.

### **Exercise 4: Design a Schedule Tool**
Plan a tool for student schedules. What parameters? What return format? How to handle errors?

**Start with Exercise 1** - it builds directly on what you learned!

---

Ready to continue with **`03_context_aware_tool_integration.ipynb`** to learn advanced context patterns?

---

## 🎯 **Ready to Practice?**

Follow this step-by-step guide to build `get_courses_by_department`. Complete each section methodically.